In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from dateutil.parser import parse

import os
import sys
import re

In [2]:
# 파일 경로
file_path = '/Users/jiminking/Desktop/LA/WebScraping/coupang_baby.csv'

# 데이터를 pandas DataFrame으로 로드
data = pd.read_csv(file_path)
scraping_date = datetime.datetime(2024, 1, 18)

# 데이터의 첫 5개 행을 출력하여 확인
print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Item Name       1020 non-null   object
 1   Discount        1020 non-null   object
 2   Original Price  1020 non-null   object
 3   Sale Price      1020 non-null   object
 4   Delivery Info   1018 non-null   object
 5   Rating Score    1020 non-null   object
 6   Total Reviews   1020 non-null   object
dtypes: object(7)
memory usage: 55.9+ KB
None


,Item Name,Discount,Original Price,Sale Price,Delivery Info,Rating Score,Total Reviews
0,"주니 유아용 자기주도 빨대컵 210ml, 화이트, 1개",28%,"35,000","25,000",내일(금) \n 도착 보장,90,119
1,"한글떼기 1~5과정 세트 전5권(개정판)(유아4~7세), 기탄출판",10%,"30,000","27,000",내일(금) \n 도착 보장,100,1692
2,"초등 미니 논술 일력 365, 서사원주니어",10%,"19,800","17,820",내일(금) \n 도착 보장,100,18
3,"락피도 철분 키즈, 90g, 1개",55%,"37,000","16,540",내일(금) \n 도착 보장,100,2905
4,"아토팜 판테놀 유아 크림, 100ml, 1개",48%,"47,000","24,110",내일(금) \n 도착 보장,100,740


In [3]:
def preprocess_column_name(df):
    """
    Preprocess column name
    Args : df - pandas DataFrame
    Return : df - pandas DataFrame
    """
    df.columns = df.columns.str.lower()  # 컬럼 이름을 소문자로 변환
    df.columns = df.columns.str.replace(" ", "_")  # 공백을 언더바로 변환
    df.columns = df.columns.str.strip()  # 컬럼 이름의 앞뒤 공백 제거
    
    return df

def split_item_name(item_name):
    """
    Split the item name by comma 
    Args : item_name : item name 
    Return : names[0] : item name before comma
    """
    names = item_name.split(',')
    return names[0]

def preprocess_price(df):
    """
    Preprocess price
    Args : dataframe
    Return : dataframe removed ',' in price 
    """
    df['original_price'] = df['original_price'].str.replace(',', '')
    df['sale_price'] = df['sale_price'].str.replace(',', '')

    # 숫자만 변환하고, 숫자가 아닌 경우 NaN으로 처리
    for col in ['original_price', 'sale_price']:  # 'sale_price'도 포함
            df[col] = df[col].str.replace(',', '')
            df[col] = pd.to_numeric(df[col], errors='coerce')
    # int로 변환 가능한 값만 int로 변환
    df['original_price'] = df['original_price'].astype('Int64')
    df['sale_price'] = df['sale_price'].astype('Int64')
    return df

def preprocess_delivery_info(delivery_info):
    if pd.isna(delivery_info):
        return delivery_info
    return ' '.join(delivery_info.replace('\n', '').split())

def determine_delivery_certainty(delivery_info):
    """"
    determine delivery certainty
    Args : delivery_info : delivery information
    Return : certainty : delivery certainty
    """
    if pd.isna(delivery_info):
        return '정보 없음'
    elif '보장' in delivery_info:
        return '확실'
    elif '예정' in delivery_info:
        return '미확실'
    else:
        return '정보 없음'

# 날짜 정보를 파싱하고, 배송 기간을 계산하는 함수
def calculate_delivery_days(delivery_info):
    try:
        if '내일' in delivery_info:
            return 1  # '내일'은 다음 날을 의미
        elif '모레' in delivery_info:
            return 2  # '모레'는 이틀 후를 의미
        elif 'No delivery info' in delivery_info:
            return None  # 배송 정보가 없는 경우
        else:
            # 날짜 파싱 (예: "1/22(월)")
            delivery_date = parse(delivery_info.split()[0], dayfirst=False, yearfirst=False)
            # 2024년으로 설정 (년도 정보가 없기 때문에)
            delivery_date = delivery_date.replace(year=2024)
            # 배송 기간 계산
            return (delivery_date - scraping_date).days
    except Exception as e:
        return None  # 파싱 오류 처리


In [5]:
print("====================Before the preprocessing====================")
print(data.info())
print(data.shape)
data = preprocess_column_name(data)
data["item_name_"] = data["item_name"].apply(split_item_name)
data = preprocess_price(data)
data['delivery_info'] = data['delivery_info'].apply(preprocess_delivery_info)
data['delivery_guarantee'] = data['delivery_info'].apply(determine_delivery_certainty)
data['delivery_days'] = data['delivery_info'].apply(calculate_delivery_days)

print("====================After the preprocessing====================")
print(data.info())
data.head()

====================Before the preprocessing====================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Item Name       1020 non-null   object
 1   Discount        1020 non-null   object
 2   Original Price  1020 non-null   object
 3   Sale Price      1020 non-null   object
 4   Delivery Info   1018 non-null   object
 5   Rating Score    1020 non-null   object
 6   Total Reviews   1020 non-null   object
dtypes: object(7)
memory usage: 55.9+ KB
None
(1020, 7)
====================After the preprocessing====================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   item_name           1020 non-null   object 
 1   discount            1020 non-null   object 
 2   origi

,item_name,discount,original_price,sale_price,delivery_info,rating_score,total_reviews,item_name_,delivery_guarantee,delivery_days
0,"주니 유아용 자기주도 빨대컵 210ml, 화이트, 1개",28%,35000,25000,내일(금) 도착 보장,90,119,주니 유아용 자기주도 빨대컵 210ml,확실,1.0
1,"한글떼기 1~5과정 세트 전5권(개정판)(유아4~7세), 기탄출판",10%,30000,27000,내일(금) 도착 보장,100,1692,한글떼기 1~5과정 세트 전5권(개정판)(유아4~7세),확실,1.0
2,"초등 미니 논술 일력 365, 서사원주니어",10%,19800,17820,내일(금) 도착 보장,100,18,초등 미니 논술 일력 365,확실,1.0
3,"락피도 철분 키즈, 90g, 1개",55%,37000,16540,내일(금) 도착 보장,100,2905,락피도 철분 키즈,확실,1.0
4,"아토팜 판테놀 유아 크림, 100ml, 1개",48%,47000,24110,내일(금) 도착 보장,100,740,아토팜 판테놀 유아 크림,확실,1.0


In [6]:
# Reordering the dataframe to have 'item_name_' column at the front
cols = list(data.columns)
if 'item_name_' in cols:
    cols.insert(0, cols.pop(cols.index('item_name_')))
    data_reordered = data[cols]
else:
    data_reordered = data  # 'item_name_' 컬럼이 없는 경우 원본 데이터 유지


In [7]:
# Removing rows with NaN values in the data_reordered dataframe
data_cleaned = data_reordered.dropna()

# Displaying the updated dataframe
data_cleaned

,item_name_,item_name,discount,original_price,sale_price,delivery_info,rating_score,total_reviews,delivery_guarantee,delivery_days
0,주니 유아용 자기주도 빨대컵 210ml,"주니 유아용 자기주도 빨대컵 210ml, 화이트, 1개",28%,35000,25000,내일(금) 도착 보장,90,119,확실,1.0
1,한글떼기 1~5과정 세트 전5권(개정판)(유아4~7세),"한글떼기 1~5과정 세트 전5권(개정판)(유아4~7세), 기탄출판",10%,30000,27000,내일(금) 도착 보장,100,1692,확실,1.0
2,초등 미니 논술 일력 365,"초등 미니 논술 일력 365, 서사원주니어",10%,19800,17820,내일(금) 도착 보장,100,18,확실,1.0
3,락피도 철분 키즈,"락피도 철분 키즈, 90g, 1개",55%,37000,16540,내일(금) 도착 보장,100,2905,확실,1.0
4,아토팜 판테놀 유아 크림,"아토팜 판테놀 유아 크림, 100ml, 1개",48%,47000,24110,내일(금) 도착 보장,100,740,확실,1.0
...,...,...,...,...,...,...,...,...,...,...
1015,랄랄라 사운드 벽보 8: 구구단,"랄랄라 사운드 벽보 8: 구구단, 키움",37%,5000,3150,내일(금) 도착 보장,90,5078,확실,1.0
1016,종이나라 투명나라풀,"종이나라 투명나라풀, 35g, 12개",53%,14400,6630,내일(금) 도착 보장,90,7445,확실,1.0
1017,지앤마 써니비 남아용 1단계 유아칫솔,"지앤마 써니비 남아용 1단계 유아칫솔, 1개입, 20개",13%,9900,8550,내일(금) 도착 보장,100,398,확실,1.0
1018,2080 브레드이발소 키즈 치약 3스텝 베리믹스향,"2080 브레드이발소 키즈 치약 3스텝 베리믹스향, 80g, 6개",57%,18900,8000,내일(금) 도착 보장,100,3662,확실,1.0
